In [691]:
import pandas as pd 
import csv
import spacy
import pt_core_news_sm
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk import ngrams

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

from geopy import geocoders  
import string
import time

In [692]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [693]:
df = pd.read_csv(r'Vagas+Catho+Gupy+Indeed+LatitudeLongitude.csv')

In [694]:
df.head()

,Unnamed: 0,Cidade,Competências,Descrição_da_empresa,Empresa,Estado,Nome_da_vaga,Termo_pesquisado,Latitude,Longitude
0,0,brasilia,o cientista de dados ou data scientist e consi...,NaN,NaN,DF,vaga de cientista de dados em brasilia - df,cientista de dados,-15.793404,-47.882317
1,1,londrina,"utilizar modelos matematicos e estatisticos, p...",NaN,NaN,PR,vaga de cientista de dados em londrina - pr,cientista de dados,-23.311288,-51.159502
2,2,rio de janeiro,interpretacao de tendencias. refino de base de...,NaN,NaN,RJ,vaga de data scientist junior/ cientista de da...,cientista de dados,-22.911014,-43.209373
3,3,rio de janeiro,"modelar problemas complexos, identificar oport...",NaN,NaN,RJ,vaga de data scientist senior/ cientista de da...,cientista de dados,-22.911014,-43.209373
4,4,rio de janeiro,realizar projetos sprint de desenvolvimento e ...,NaN,NaN,RJ,vaga de cientista de dados (aut) em rio de jan...,cientista de dados,-22.911014,-43.209373


In [695]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2278 entries, 0 to 2277
Data columns (total 10 columns):
Unnamed: 0              2278 non-null int64
Cidade                  2278 non-null object
Competências            2278 non-null object
Descrição_da_empresa    541 non-null object
Empresa                 830 non-null object
Estado                  1509 non-null object
Nome_da_vaga            2278 non-null object
Termo_pesquisado        2278 non-null object
Latitude                2278 non-null float64
Longitude               2278 non-null float64
dtypes: float64(2), int64(1), object(7)
memory usage: 178.0+ KB


In [696]:
df.drop_duplicates(subset ="Competências", inplace = True) 

In [697]:
df = df.reset_index(drop=True)

In [698]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [699]:
df.groupby(['Termo_pesquisado']).count()

,Cidade,Competências,Descrição_da_empresa,Empresa,Estado,Nome_da_vaga,Latitude,Longitude
Termo_pesquisado,,,,,,,,
analista de dados,383,383,186,269,115,383,383,383
analista de negocios,205,205,145,205,0,205,205,205
analista de negocios,654,654,0,0,654,654,654,654
business analytics,17,17,4,5,12,17,17,17
business inteligence,8,8,3,4,4,8,8,8
cientista de dados,104,104,12,67,88,104,104,104
data analytics,27,27,19,27,2,27,27,27
data analytics,44,44,0,0,44,44,44,44
data science,62,62,8,12,52,62,62,62


In [700]:
termo_pesquisado = [
#                    'Cientista+de+dados',
#                    'Data+Science',
                    'data analytics',
                    'engenheiro de machine learning',
                    'analista de dados',
                    'engenheiro de dados',
                    'business inteligence',
                    'business analytics',
                    'analista de negocios',
                    'inteligencia de negocios'
                    ]

In [701]:
punctuations = string.punctuation
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [702]:
words = nlp(u'        sera garantir acompanhar acordo sob orientacao estar cursando lista equipamentos memorias calculo industriais entregaveis disciplina tais requerimentos boas trabalho area mercado beneficios buscamos profissinais vai fazer parte encontrar desejaveis cliente provendo dono areas dado dados desejavel de a o que e do da em um para é com não uma os no se na por mais as dos como mas foi ao ele das tem à seu sua ou ser quando muito há nos já está eu também só pelo pela até isso ela entre era depois sem mesmo aos ter seus quem nas me esse eles estão você tinha foram essa num nem suas meu às minha têm numa pelos elas havia seja qual será nós tenho lhe deles essas esses pelas este fosse dele tu te vocês vos lhes meus minhas teu tua teus tuas nosso nossa nosso nossas dela delas esta estes estas aquele aquela aqueles aquelas isto aquilo estou está estamos estão estive esteve estivemos estiveram estava estávamos estavam estivera estivéramos esteja estejamos estejam estivesse estivéssemos estivessem estiver estivermos estiverem hei há havemos hão houve houvemos houveram houvera houvéramos haja hajamos hajam houvesse houvéssemos houvessem houver houvermos houverem houverei houverá houveremos houverão houveria houveríamos houveriam sou somos são era éramos eram fui foi fomos foram fora fôramos seja sejamos sejam fosse fôssemos fossem for formos forem serei será seremos serão seria seríamos seriam tenho tem temos tém tinha tínhamos tinham tive teve tivemos tiveram tivera tivéramos tenha tenhamos tenham tivesse tivéssemos tivessem tiver tivermos tiverem terei terá teremos terão teria teríamos teriam')
#words_sem_de = nlp(u'a o que e do da em um para é com não uma os no se na por mais as dos como mas foi ao ele das tem à seu sua ou ser quando muito há nos já está eu também só pelo pela até isso ela entre era depois sem mesmo aos ter seus quem nas me esse eles estão você tinha foram essa num nem suas meu às minha têm numa pelos elas havia seja qual será nós tenho lhe deles essas esses pelas este fosse dele tu te vocês vos lhes meus minhas teu tua teus tuas nosso nossa nosso nossas dela delas esta estes estas aquele aquela aqueles aquelas isto aquilo estou está estamos estão estive esteve estivemos estiveram estava estávamos estavam estivera estivéramos esteja estejamos estejam estivesse estivéssemos estivessem estiver estivermos estiverem hei há havemos hão houve houvemos houveram houvera houvéramos haja hajamos hajam houvesse houvéssemos houvessem houver houvermos houverem houverei houverá houveremos houverão houveria houveríamos houveriam sou somos são era éramos eram fui foi fomos foram fora fôramos seja sejamos sejam fosse fôssemos fossem for formos forem serei será seremos serão seria seríamos seriam tenho tem temos tém tinha tínhamos tinham tive teve tivemos tiveram tivera tivéramos tenha tenhamos tenham tivesse tivéssemos tivessem tiver tivermos tiverem terei terá teremos terão teria teríamos teriam')
words = [token.orth_ for token in words if not token.is_punct]
stopwords = frozenset(words)

In [703]:
words_ingles = nlp(u"bairesdev chain inteligence or knowledge is seeking a data scientist tech lead to work with some of our most important clients in usa and europe. be part of an extraordinary opportunity to transform the higher experience! what we need? bairesdev is looking for a seasoned tech lead to join a stellar team working on it marketplace industry across the world. you will work with the developer and deliver & maintain critical software to serve clients, front office, and the general business community to ensure excellent customer service experiences for all internal and external customers. you will have the opportunity to work on projects for companies that make a difference in each category. what we can offer you: one of our core values at bairesdev is to recognise and reward our staff as our greatest asset. we realise that it\'s our people who have built our outstanding reputation and that we will only maintain our leading position in the it world by continuing to attract and retain talented and motivated people. if you choose to come and work with us, you will find that we offer: excellent benefits and salary. a welcoming and inclusive environment. extensive development opportunities. stunning surroundings. remote work if this sounds great to you, please apply now. experience working with data scientist technology. experience in software engineering. bairesdev is looking for a data scientist ssr/sr developer to work for a very important client in usa. this is a great opportunity for someone to be part of a rapid growth of a close knit team. this job offer is for full time and long term engagement, and you are allowed to work from your home. we love the rockstars, the self-taught geniuses, the niche specialists who know things almost no one knows. we love heads-down producers, the imaginative renegades, the go-to veterans who know what they know inside out. what we can offer you. one of our core values at bairesdev is to recognize and reward our staff as our greatest asset. we realise that it's our people who have built our outstanding reputation and that we will only maintain our leading position in the it world by continuing to attract and retain talented and motivated people. if you choose to come and work with us, you will find that we offer: excellent benefits and salary. a welcoming and inclusive environment. extensive development opportunities the minimum requirements are: data scientist experience working as a developer. proficient with analysis, troubleshooting, and problem solving. hands-on experience with managing data loads and data quality a plus. advanced english is mandatory")
words_ingles = [token.orth_ for token in words_ingles if not token.is_punct]
stopwords_ingles = frozenset(words_ingles)

In [704]:
def get_first(iterable, default=None):
    if iterable:
        for item in iterable:
            return item
    return default

In [705]:
nlp = spacy.load('pt')

In [707]:
for i in range(0, len(termo_pesquisado)):
    termo = termo_pesquisado[i]
    print (termo)
    
    df_science = df.loc[(df['Termo_pesquisado'] == termo)]
    print (df_science)

    df_resultado = df_science.copy()
    
    tokenization = []
    for i in df_science.Competências:
        doc = nlp(i)
        tokens = [token.orth_ for token in doc if not token.is_punct]
        tokens = [token for token in tokens if (token not in stopwords and token not in punctuations and token not in stopwords_ingles)]
        tokenization += (tokens)
        
    ngram_counts1 = Counter(ngrams(tokenization, 1))
    ngram_counts2 = Counter(ngrams(tokenization, 2))
    
    ngram_countsAll = ngram_counts1.most_common(10) + ngram_counts2.most_common(10)
   
    for j in ngram_countsAll:
        Item = get_first(j)
        ItemString = ' '.join(Item)
        print (ItemString)
        if ItemString == 'r': 
              ItemString = ' ' + ItemString + ' ' 
        df_resultado[ItemString] = df_resultado['Competências'].apply(lambda x: 1 if ItemString in x else 0)
    
#    df_resultado.drop(['Unnamed: 0', 'index'], axis=1, inplace=True)
    arquivo = 'resultadov2 ' + termo + '.csv'
    df_resultado.to_csv(arquivo)
    print (df_resultado)

data analytics
              Cidade                                       Competências  \
1013       sao paulo  principais responsabilidades- desenvolvimento ...   
1014       sao paulo  principais responsabilidades- atuacao com area...   
1015       sao paulo  principais atividades:desenho e manutencao da ...   
1016       sao paulo  job descriptiondesign e gestao de data analyti...   
1017       sao paulo  requisitos:ensino superior completoconheciment...   
1018       sao paulo  objetivos:contribuir para que a organizacao al...   
1019       sao paulo  atividade:entregar servicos profissionais basi...   
1020       sao paulo  atividades:responsavel pelo mapeamento de todo...   
1021       sao paulo  principais atividades:analisar demonstracoes f...   
1022       sao paulo  responsabilidades- organizar dados e gerar rel...   
1023       sao paulo  vp: marketingarea: marketing digitallocal: ber...   
1024       sao paulo  empresa: votorantim cimentos s/ags-31escolarid...   
1025      

experiencia
analytics
gestao
processos
projetos
conhecimento
analise
ferramentas
relatorios
solucoes
power bi
ciencia computacao
tecnologia informacao
superior completo
web analytics
auditoria interna
auditoria continua
google analytics
escolhas tecnicas
especificacao funcional
              Cidade                                       Competências  \
1013       sao paulo  principais responsabilidades- desenvolvimento ...   
1014       sao paulo  principais responsabilidades- atuacao com area...   
1015       sao paulo  principais atividades:desenho e manutencao da ...   
1016       sao paulo  job descriptiondesign e gestao de data analyti...   
1017       sao paulo  requisitos:ensino superior completoconheciment...   
1018       sao paulo  objetivos:contribuir para que a organizacao al...   
1019       sao paulo  atividade:entregar servicos profissionais basi...   
1020       sao paulo  atividades:responsavel pelo mapeamento de todo...   
1021       sao paulo  principais atividades:an

learning
machine
experiencia
tecnicas
solucoes
python
deep
conhecimentos
sql
analise
machine learning
tecnicas machine
sao paulo
deep understanding
learning visao
visao computacional
maquinas eletricas
python r
indicadores definicao
extratores automatizados
              Cidade                                       Competências  \
131   jaragua do sul  implementacao de sistema de apoio a decisao pa...   
132        sao paulo  transformacao dos dados de acordo com as neces...   
133        sao paulo  o engenheiro de dados sera responsavel por rea...   
134        sao paulo  selecionamos profissional para atuar como enge...   
135        sao paulo  about escale escale is a rapidly growing custo...   
136         campinas  orientar e trabalhar em conjunto com o time de...   
1038       sao paulo  o analista sera responsavel pela estruturacao ...   
1039       sao paulo  software engineer (full stack)atividades: prof...   

                                   Descrição_da_empresa           

conhecimento
experiencia
sistemas
desenvolvimento
analise
banco
processos
sql
superior
relatorios
superior completo
sql server
ensino superior
sistemas informacao
tecnologia informacao
analise sistemas
ciencia computacao
banco sql
experiencia desenvolvimento
power bi
                       Cidade  \
137                    manaus   
138                  salvador   
139                  salvador   
140                    recife   
141                 fortaleza   
142                  brasilia   
143                  brasilia   
144                  sao luis   
145            belo horizonte   
146            belo horizonte   
147            belo horizonte   
148            belo horizonte   
149              juiz de fora   
150            belo horizonte   
151                  curitiba   
152                  curitiba   
153                   maringa   
154                    recife   
155                    recife   
156                    recife   
157            rio de janeiro   
158   

projetos
engenharia
experiencia
desenvolvimento
servicos
analise
conhecimento
automacao
projeto
equipe
linguagens programacao
experiencia projetos
superior completo
engenharia mecanica
definicoes projeto
projeto praticas
projetos mineracao
desenvolvimento projetos
normas aplicaveis
aplicaveis definicoes
              Cidade                                       Competências  \
1307       sao paulo  se voce manja de programacao web, tem interess...   
1308       sao paulo  atividadeselaboracao de memorial descritivo de...   
1309       sao paulo  procuramos estudantes para vagas de engenharia...   
1310       sao paulo  atividades:criar e estruturar area de devops n...   
1311       sao paulo  responsabilidades:estruturar e implementar cod...   
1312   nao informado  empresa multinacional busca profissional para ...   
1313       sao paulo  conhecimentos necessarios:*         arquitetur...   
1314       sao paulo  empresa multinacional busca engenheiro automac...   
1315       sao paulo



informacao
experiencia
processos
conhecimento
gestao
analise
sistemas
desenvolvimento
tecnologia
tecnologia informacao

 conhecimento
solucoes ti
informacao digitalizacao
sql server
definicao processos
ferramentas bi
conhecimento analise

 experiencia
superior completo
                   Cidade                                       Competências  \
278        belo horizonte  atuar com  modelagem de dw e em tunning de ban...   
279              cascavel  o profissional conduzira o processo de descobr...   
280             sao paulo  responder duvidas e fornecer detalhes de regra...   
281   sao jose dos campos  desenvolvimento de sistemas plataforma web e a...   
1351            sao paulo  pessoas talentosas fazem da schneider electric...   
1352       rio de janeiro  experiencia em (apenas highlights): desejavel ...   
1353             sorocaba  vaga para atuar na regiao norte. atuacao a dis...   
1354        florianopolis  1.      pre-requisitos obrigatorio:voce precis...   

       

analytics
analise
conhecimento


modelagem
projetos
relatorios
bi
desenvolvimento
apoiar
machine learning
relatorios dashboards
tecnologias analytics
intelligence analytics
power bi
matematica engenharia
google analytics
dimensoes metricas
ferramentas intelligence
formacao superior
                      Cidade  \
282                 brasilia   
283           belo horizonte   
284                 curitiba   
285                sao paulo   
286                sao paulo   
287                sao paulo   
288                sao paulo   
289                sao paulo   
290                  barueri   
291                sao paulo   
292                sao paulo   
293   vargem grande paulista   
1355               sao paulo   
1356               sao paulo   
1357               sao paulo   
1358               guarulhos   
1359                campinas   

                                           Competências  \
282   levantamento de atividades junto as areas; ana...   
283   somos referencia

processos
gestao
projetos
desenvolvimento
experiencia
negocio
analise
conhecimento
atividades
superior
superior completo
completo administracao
administracao economia
ensino superior
formacao superior
administracao empresas
dar suporte
recursos humanos
graduacao completa
sistemas informacao
                     Cidade  \
1360              sao paulo   
1361              sao paulo   
1362              sao paulo   
1363              sao paulo   
1364              sao paulo   
1365              sao paulo   
1366              sao paulo   
1367              sao paulo   
1368              sao paulo   
1369              sao paulo   
1370              sao paulo   
1371              sao paulo   
1372              sao paulo   
1373              sao paulo   
1374              sao paulo   
1375              sao paulo   
1376              sao paulo   
1377              sao paulo   
1378              sao paulo   
1379              sao paulo   
1380              sao paulo   
1381              sao paul

inteligencia
experiencia
marketing
negocio
conhecimento
gestao
desenvolvimento
clientes
negocios
vendas
superior completo
ensino superior
inteligencia negocios
negocio inteligencia
power bi
completo administracao
administracao empresas
inteligencia negocio
marketing digital
tecnologia informacao
                     Cidade  \
948                brasilia   
949                brasilia   
950                 vitoria   
951                 goiania   
952                curitiba   
953               guarulhos   
954               sao paulo   
955               sao paulo   
956               sao paulo   
957               sao paulo   
958               sao paulo   
959               sao paulo   
960                 barueri   
961                 barueri   
962               sao paulo   
963               sao paulo   
964               sao paulo   
965               sao paulo   
966               sao paulo   
967             santo andre   
968               sao paulo   
969               sao